Model ResNet

https://www.pluralsight.com/guides/introduction-to-resnet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy

import os
import sys
sys.path.insert(1, 'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data_pipeline')
import os
from data_sampler import WeightedSampler
from dataset import CARLADataset#, CARLADatasetMultiProcessing
from torch.utils.data import DataLoader

In [2]:
class MyResnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torchvision.models.resnet18(pretrained=True)
        num_ftrs = self.net.fc.in_features
        """
        self.net.fc = nn.Sequential(
            nn.Linear(512, 3), #nbr of outputs self.net.fc.in_features
            nn.Tanh()
        )
        """
        self.net.fc = nn.Identity()
        self.thr_head = nn.Linear(num_ftrs, 1)
        self.brk_head = nn.Linear(num_ftrs, 1)
        self.str_head = nn.Linear(num_ftrs, 1)
        
        self.net = self.net.cuda() if device else self.net
        self.net.fc = self.net.fc.cuda() if device else self.net.fc
        """
        self.thr_head = self.thr_head.cuda() if device else self.thr_head
        self.brk_head = self.brk_head.cuda() if device else self.brk_head
        self.str_head = self.str_head.cuda() if device else self.str_head
        """

    def forward(self, data_):
        x = self.net(data_)
        return self.thr_head(x), self.str_head(x), self.brk_head(x)

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = MyResnet()
net = net.cuda() if device else net
net

C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MyResnet(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

Data Loaders etc


In [4]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }

train_dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=None)
weighted_sampler = WeightedSampler(dataset=train_dataset)

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Town10HD_Scenario10_route9_11_28_17_59_39 has varying number of data files among input folders. It got discarded from the dataset.


In [5]:
def normalize_img(tensor):
    v_min, v_max = tensor.min(), tensor.max()
    new_min, new_max = 0, 1
    v_p = (tensor - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    return v_p

Training

In [6]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [13]:
%%time

n_epochs = 1
#print_every = 10
valid_loss_min = np.Inf
val_loss = []
train_loss = []
total_step = len(train_dataloader)

run = True

for epoch in range(1, n_epochs+1):
    
    running_loss = 0.0
    print(f'Epoch {epoch}\n')
    
    #data: (['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])
    
    # Work through batches
    for batch_idx, data in enumerate(train_dataloader):

        # further preprocessing
        data_ = normalize_img(data["rgb"].transpose(1,4)[:,:,:,:,0]).float() # HOTFIX
        #target_ = torch.stack([,,])[:,:,0].transpose(0,1).float() # HOTFIX
        
        #print(data["throttle"].shape)
        
        data["throttle"]=data["throttle"].float()
        data["steer"]=data["steer"].float()
        data["brake"]=data["brake"].float()
        
        #print(target_.shape)
        #print(target_.shape[0])
        if data["throttle"].shape[0] != 16: # HOTFIX
            run = False
            break

        # move to GPU
        data_ = data_.to(device)
        
        data["throttle"] = data["throttle"].to(device)
        data["steer"] = data["steer"].to(device)
        data["brake"] = data["brake"].to(device)
        
        # compute outputs
        optimizer.zero_grad()
        
        outputs = net(data_)
        output_throttle = outputs[0].to(device)
        output_steer = outputs[1].to(device)
        output_brake = outputs[2].to(device)
        
        loss_throttle = criterion(output_throttle, data["throttle"].float())
        loss_steer = criterion(output_steer, data["steer"].float())
        loss_brake = criterion(output_brake, data["brake"].float())
        
        
        loss = sum([loss_throttle, loss_steer, loss_brake])/3
        
        # Backprop
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()
        if (batch_idx) % batch_size == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))

    if not run: # HOTFIX
        continue
        
    # Epoch finished, evaluate network and save if network_learned
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f},')
    batch_loss = 0

    
    """
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (test_dataloader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
        val_loss.append(batch_loss/len(test_dataloader))
        #network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, \n')

        
        if False:#network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    """

    net.train()
    

Epoch 1

Epoch [1/1], Step [0/126], Loss: 0.4959
Epoch [1/1], Step [16/126], Loss: 0.4320
Epoch [1/1], Step [32/126], Loss: 0.3972
Epoch [1/1], Step [48/126], Loss: 0.3692
Epoch [1/1], Step [64/126], Loss: 0.2838
Epoch [1/1], Step [80/126], Loss: 0.3582
Epoch [1/1], Step [96/126], Loss: 0.3118
Epoch [1/1], Step [112/126], Loss: 0.3554
Wall time: 3min 12s


Test predictions

In [ ]:
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(next(iter(test_dataloader)).keys())
idx, X = next(enumerate(test_dataloader))

In [ ]:
data_ = normalize_img(X["rgb"].transpose(1,4)[:,:,:,:,0]).float().to(device)
target_ = torch.stack([X["steer"],X["throttle"],X["brake"]])[:,:,0].transpose(0,1).float()

In [ ]:
outputs_ = net(data_)

In [ ]:
print(outputs_[:,:])

In [ ]:
print(target_)

save & load

In [ ]:
torch.save(net, 'rgb_resnet.pth')

In [ ]:
net = torch.load('rgb_resnet.pth')

Time test

In [29]:
# ohne preprocessing ca 16-17 sekunden. Mit preprocessing ca 35 sekunden
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
at = time.time()
for batch_idx, data in enumerate(train_dataloader):
    data_ = normalize_img(data["rgb"].transpose(1,4)[:,:,:,:,0]).float()
    target_ = torch.stack([data["speed"],data["steer"],data["throttle"],data["brake"]])[:,:,0].transpose(0,1).float()
et = time.time()
print(et-at)


36.362138986587524
